In [1]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever
from transformers import AutoTokenizer, AutoModel

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"D:\python_project\chaekchecklab\data\merge_001001025006.csv", index_col=0)

In [3]:
df_test = df.dropna(subset=['intro'])
df_test.head()

,title,full_name,book_url,authors,publisher,author_list,item_infos,cates,intro
4,한국의 장기통계 2,한국의 장기통계 2 [ 반양장 ],https://www.yes24.com/Product/Goods/58259617,"김낙년,박기주,박이택,차명수 공저",해남,"[['김낙년'], ['박기주'], ['박이택'], ['차명수']]","['2018년 01월 18일', '반양장', '616쪽 | 227*303mm', '...",['국내도서>경제 경영>총람/연감'],"일반인은 물론, 연구자들도 자신의 전공을 넘어 다른 분야에서 원하는 통계를 찾기가 ..."
5,한국의 장기통계 1,한국의 장기통계 1 [ 반양장 ],https://www.yes24.com/Product/Goods/58259614,"김낙년,박기주,박이택,차명수 공저",해남,"[['김낙년'], ['박기주'], ['박이택'], ['차명수']]","['2018년 01월 18일', '반양장', '616쪽 | 227*303mm', '...",['국내도서>경제 경영>총람/연감'],"일반인은 물론, 연구자들도 자신의 전공을 넘어 다른 분야에서 원하는 통계를 찾기가 ..."
8,2023 생활물가총람,2023 생활물가총람,https://www.yes24.com/Product/Goods/123877945,한국물가정보 조사부 편,한국물가정보(단행본),[['한국물가정보 조사부']],"['2023년 11월 29일', '320쪽 | 188*257*30mm', '9791...",['국내도서>경제 경영>총람/연감'],"고물가 시대 속, 현명한 소비를 위한 지표[생활물가총람]은 (사)한국물가정보에서 매..."
10,국내외 디스플레이 및 소재/부품/장비 시장 분석과 기술 개발 동향,국내외 디스플레이 및 소재/부품/장비 시장 분석과 기술 개발 동향,https://www.yes24.com/Product/Goods/106490781,미래산업분석연구회 저,리서치컴퍼니,[['미래산업분석연구회']],"['2022년 01월 21일', '822쪽 | 210*297*47mm', '9791...",['국내도서>경제 경영>총람/연감'],"*머릿말 2021년 디스플레이 산업은 상, 하반기 분위기가 너무나 상이했던 한 해로..."
12,2022 한경기업총람,"2022 한경기업총람 [ 구성 : CD 1장 + USB 1장 + 도서, 양장 ]",https://www.yes24.com/Product/Goods/102831550,NICE평가정보 편,한국경제신문사(한경비피),"[['NICE평가정보', '277929']]","['2021년 07월 22일', '양장', '520쪽 | 188*257*35mm',...",['국내도서>경제 경영>총람/연감'],"최고의 검색 및 기업마케팅 솔루션, ≪2022 한경기업총람≫ 발간!〈최신기업DB 검..."


In [4]:
# 1. FAISS 기반의 Document Store와 Retriever 설정
# document_store = FAISSDocumentStore(embedding_dim=768, faiss_index_factory_str="Flat")
document_store = FAISSDocumentStore(embedding_dim=384, faiss_index_factory_str="Flat")

In [5]:
retriever = EmbeddingRetriever(document_store=document_store, embedding_model="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\jjkha\anaconda3\envs\recsys\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
# 2. 문서들을 임베딩하여 저장 (document embedding)
docs = df_test[['intro']].to_dict()['intro']

# Retriever를 이용해 문서 임베딩 계산 및 저장
document_store.write_documents([{'idx':i, "content": docs[i]} for i in docs])
document_store.update_embeddings(retriever)

Writing Documents: 10000it [00:03, 3089.30it/s]           


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/609 [00:00<?, ? docs/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Documents Processed: 10000 docs [02:12, 75.39 docs/s]          


In [28]:
# 3. 임베딩 생성 (토크나이저와 모델 사용)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# 문서 임베딩 계산
q_docs = ["한국 프랜차이즈", "미국 자동차"]
doc_embeddings = np.array([embed_text(doc) for doc in q_docs])

# 4. 5개의 문서 임베딩을 평균내어 대표 임베딩 계산
representative_embedding = np.mean(doc_embeddings, axis=0)

# 5. 대표 임베딩을 사용한 검색 (MMR 방식)
# Retriever에서 유사한 문서 검색
# similar_docs = retriever.retrieve(query=None, query_emb=representative_embedding, top_k=3, use_mmr=True, diversity=0.7)
similar_docs = document_store.query_by_embedding(representative_embedding, top_k=10)

c:\Users\jjkha\anaconda3\envs\recsys\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
# 검색 결과 출력
for doc in similar_docs:
    print(doc.meta, doc.content)

{'idx': 614, 'vector_id': '351'} 이 책은 한국무역전략연구원의 한국무역연감을 다룬 정부간행물이다.
{'idx': 613, 'vector_id': '131'} 이 책은 한국문화사회연구원의 한국문화예술연감이다.
{'idx': 392, 'vector_id': '326'} 한국전자산업진흥회에서 파악하고 있는 전자계측기의 시장규모와 기술현황을 담은 책이다.
{'idx': 216, 'vector_id': '240'} 대한민국 학원에 대한 연간 정보를 담았다. 현재 학원 산업의 현황과 추세, 그리고 앞으로의 계획과 발전방향에 대해 자세히 기록한 자료집이다.
{'idx': 611, 'vector_id': '219'} ▶ 이 책은 한국조경산업연구원의 한국조경산업연감이다.
